# Grabbing Frames from an SD Card

The most basic example, point us at an SD card and start grabbin!

In [1]:
from pprint import pprint
from pathlib import Path
from miniscope_io.io import SDCard
from miniscope_io.formats import WireFreeSDLayout

The `WireFreeSDConfig` object is an instance of the `sdcard.SDLayout` class. The `SDLayout`
class is a generic way of declaring how data is laid out on an SD card, and the WireFree layout
is one instance of that! This layer of abstraction lets us write reusable code for grabbing
frames, and also gives us a way of storing, communicating, and reasoning about SD card layouts
in a way that's a little more flexible than hardcoding constants. In a future version we
probably want to be more explicit about how to map individual sectors of the model to the
SD card so we can do stuff like random seek and whatnot, but this will do for now!

It looks like this:

In [3]:
pprint(WireFreeSDLayout.dict(), sort_dicts=False)

{'sectors': {'header': 1023, 'config': 1024, 'data': 1025, 'size': 512},
 'write_key0': 226277911,
 'write_key1': 226277911,
 'write_key2': 226277911,
 'write_key3': 226277911,
 'word_size': 4,
 'header': {'gain': 4, 'led': 5, 'ewl': 6, 'record_length': 7, 'fs': 8},
 'config': {'width': 0,
            'height': 1,
            'fs': 2,
            'buffer_size': 3,
            'n_buffers_recorded': 4,
            'n_buffers_dropped': 5},
 'buffer': {'length': 0,
            'linked_list': 1,
            'frame_num': 2,
            'buffer_count': 3,
            'frame_buffer_count': 4,
            'write_buffer_count': 5,
            'dropped_buffer_count': 6,
            'timestamp': 7,
            'data_length': 8}}


Using the SDCard class with the SDLayout class is straightforward, but since I don't have a
data sample I can't actually test that it works right now!

In the future once I get an image we would want to make some jupyter widget to select possible drives,
but for now i'll just hardcode it as a string for the sake of an example. The `SDCard` class has a
`check_valid` method that looks for the `WRITE_KEY`s as in the prior notebook, so we could also
make that into a classmethod and just automatically find the right drive that way.

In [ ]:
drive = Path('/') / 'using' / 'OS' / 'independent' / 'paths'
sd = SDCard(drive=drive, layout = WireFreeSDLayout)

for frame in sd.reader():
    # do something with the frame!
    pass

# or we can use it outside of a loop like this
# reader = sd.reader()
# frame = next(reader)